# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

# But, I only did for llama3.2 since don't have API subscription

In [1]:
# imports

import os
import json
from dotenv import load_dotenv #no nned for ollama
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
MODEL = MODEL_LLAMA

In [5]:
# set up environment
system_prompt = """
You are an expert Python programmer and teacher. Your task is to carefully explain what a given Python code snippet does. 
Explain both the mechanics (how the code works step by step) and the rationale (why someone would write it this way). 
Use clear, concise language suitable for someone familiar with Python but not necessarily advanced. 
Highlight any Python-specific constructs or idioms used in the code.

Do not write unrelated text; focus only on explaining the code.
"""


In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
# Get gpt-4o-mini to answer, with streaming

In [8]:
# Get Llama 3.2 to answer
response = ollama.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
)
print(response.choices[0].message.content)


Let's break down this code step by step:

1. `yield from`: This is a Python 3.3+ idiom that allows us to expand a single iteration over multiple nested iterators at once. However, in this specific case, the outer iterator is not present, so it will be used only as a keyword argument to the function.

2. `{book.get("author") for book in books if book.get("author")}`: This is a dictionary comprehension combined with an iterator (the `for` loop) that filters and transforms data using another iterator (another `for` loop).

   - The inner `for...if...in` statement filters the dictionaries (`books`) based on the condition `book.get("author")`, which means only books whose value for `"author"` key is available will be included in the result.
   
     Since dictionary keys are used here, it's assumed that `book` variable contains a dictionary where "author" is a string. 

   - Similarly,  `{...for book}` creates an iterator (iterator object) which yields each element from this list.


So how 